In [5]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.3 MB/s eta 0:00:00


In [8]:
import os
import sys
import zipfile
import glob
import time
import gc
import re
import json
import math
from pathlib import Path
from typing import Any, List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Скрываем TensorFlow логи
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Импорты ML
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import optuna

# Обработка ошибок импортов
try:
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except ImportError:
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    print("⚠️  sentence-transformers не установлен. Используем TF-IDF вместо BERT.")

try:
    import implicit
    IMPLICIT_AVAILABLE = True
except ImportError:
    IMPLICIT_AVAILABLE = False
    print("⚠️  implicit не установлен. Пропускаем ALS фичи.")

try:
    from catboost import CatBoostRegressor
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("⚠️  catboost не установлен. Используем только LightGBM.")

try:
    import torch
    TORCH_AVAILABLE = True
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
except ImportError:
    TORCH_AVAILABLE = False
    DEVICE = "cpu"

# Для работы с пропусками
from sklearn.impute import SimpleImputer

# NLTK для обработки текста
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Фиксация случайности
def seed_everything(seed=42):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if TORCH_AVAILABLE:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

seed_everything(42)

# === КОНСТАНТЫ ===
class constants:
    COL_USER_ID = "user_id"
    COL_BOOK_ID = "book_id"
    COL_TARGET = "rating"
    COL_HAS_READ = "has_read"
    COL_TIMESTAMP = "timestamp"
    COL_PREDICTION = "rating_predict"

    COL_GENDER = "gender"
    COL_AGE = "age"
    COL_AUTHOR_ID = "author_id"
    COL_PUBLICATION_YEAR = "publication_year"
    COL_LANGUAGE = "language"
    COL_PUBLISHER = "publisher"
    COL_AVG_RATING = "avg_rating"
    COL_DESCRIPTION = "description"
    COL_TITLE = "title"

    # Жанры
    COL_GENRE_ID = "genre_id"

    F_USER_MEAN_RATING = "user_mean_rating"
    F_USER_RATINGS_COUNT = "user_ratings_count"
    F_BOOK_MEAN_RATING = "book_mean_rating"
    F_BOOK_RATINGS_COUNT = "book_ratings_count"
    F_AUTHOR_MEAN_RATING = "author_mean_rating"
    F_BOOK_GENRES_COUNT = "book_genres_count"

    MISSING_CAT_VALUE = "unknown"
    MISSING_NUM_VALUE = -1
    PREDICTION_MIN_VALUE = 0
    PREDICTION_MAX_VALUE = 10

class config:
    RANDOM_STATE = 42
    TARGET = constants.COL_TARGET

    # Разделение данных
    TEMPORAL_SPLIT_RATIO = 0.85

    # Параметры остановки
    EARLY_STOPPING_ROUNDS = 100

    # Файлы
    MODEL_FILENAME = "super_model"
    FEATURE_IMPORTANCE_PATH = "feature_importance_final.csv"

    # Параметры обработки текста
    USE_BERT_EMBEDDINGS = False  # Отключаем BERT для стабильности
    TFIDF_MAX_FEATURES = 300
    TFIDF_MIN_DF = 3
    TFIDF_MAX_DF = 0.9
    TFIDF_NGRAM_RANGE = (1, 2)

    # Графовые фичи
    ENABLE_GRAPH_FEATURES = False  # Отключаем ALS для стабильности

    # Ансамблирование
    ENSEMBLE_MODELS = False  # Отключаем ансамбль для стабильности

    # Продвинутые фичи
    ENABLE_TARGET_ENCODING = True
    ENABLE_TEMPORAL_FEATURES = True
    ENABLE_ADVANCED_INTERACTIONS = True
    ENABLE_CLUSTER_FEATURES = False  # Отключаем кластеризацию для стабильности

    # Категориальные фичи
    CAT_FEATURES = [
        constants.COL_USER_ID,
        constants.COL_BOOK_ID,
        constants.COL_GENDER,
        constants.COL_AUTHOR_ID,
        constants.COL_LANGUAGE,
        constants.COL_PUBLISHER,
    ]

    # Параметры LightGBM
    LGB_PARAMS = {
        "objective": "rmse",
        "metric": "rmse",
        "n_estimators": 5000,
        "learning_rate": 0.01,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.9,
        "bagging_freq": 1,
        "lambda_l1": 0.5,
        "lambda_l2": 1.0,
        "num_leaves": 63,
        "min_child_samples": 30,
        "verbose": -1,
        "n_jobs": -1,
        "seed": RANDOM_STATE,
        "boosting_type": "gbdt",
        "device": "cpu",  # Используем CPU для стабильности
    }

    # Оптимизация
    MEMORY_LIMIT_GB = 10

# === УТИЛИТЫ ===
def find_file(filename):
    """Находит файл в текущей директории и подпапках"""
    for root, dirs, files in os.walk("."):
        if filename in files:
            return os.path.join(root, filename)
    raise FileNotFoundError(f"Не могу найти {filename}!")

def safe_to_int(series, fill_value=0):
    """Безопасная конвертация в целое число с обработкой NaN"""
    return pd.to_numeric(series, errors='coerce').fillna(fill_value).astype(np.int8)

def reduce_mem_usage(df: pd.DataFrame, aggressive=False) -> pd.DataFrame:
    """Продвинутая оптимизация памяти с обработкой нечисловых типов"""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"📊 Начальный размер памяти: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        # Пропускаем нечисловые и временные типы
        if col_type == 'object' or str(col_type).startswith(('datetime', 'period', 'category')):
            continue

        if col_type.name != "category" and "datetime" not in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if aggressive and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].round(5 if aggressive else 3).astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"✅ Оптимизированный размер: {end_mem:.2f} MB (сэкономлено {100*(start_mem-end_mem)/start_mem:.1f}%)")
    return df

def temporal_split_by_date(df: pd.DataFrame, split_ratio: float = 0.8) -> Tuple[pd.Series, pd.Series]:
    """Разделение по времени с защитой от ошибок"""
    try:
        # Сортируем по времени, заполняем пропуски медианой
        if constants.COL_TIMESTAMP not in df.columns:
            print("⚠️  Нет timestamp - создаем искусственный")
            df[constants.COL_TIMESTAMP] = pd.date_range(
                start='2020-01-01',
                periods=len(df),
                freq='D'
            )
        else:
            if not pd.api.types.is_datetime64_any_dtype(df[constants.COL_TIMESTAMP]):
                df[constants.COL_TIMESTAMP] = pd.to_datetime(
                    df[constants.COL_TIMESTAMP],
                    errors='coerce',
                    infer_datetime_format=True,
                    dayfirst=True
                )

            # Заполняем пропуски
            if df[constants.COL_TIMESTAMP].isnull().any():
                median_time = df[constants.COL_TIMESTAMP].median()
                null_count = df[constants.COL_TIMESTAMP].isnull().sum()
                print(f"⚠️  Заполняем {null_count} пропусков в timestamp медианой: {median_time}")
                df[constants.COL_TIMESTAMP] = df[constants.COL_TIMESTAMP].fillna(median_time)

        df_sorted = df.sort_values(by=constants.COL_TIMESTAMP)
        split_idx = int(len(df_sorted) * split_ratio)

        train_mask = pd.Series(False, index=df.index)
        val_mask = pd.Series(False, index=df.index)

        train_mask.loc[df_sorted.index[:split_idx]] = True
        val_mask.loc[df_sorted.index[split_idx:]] = True

        return train_mask, val_mask

    except Exception as e:
        print(f"❌ Ошибка при разделении по времени: {str(e)}")
        # Резервный вариант - случайное разделение
        print("⚠️  Используем случайное разделение как резервный вариант")
        train_mask = np.random.rand(len(df)) < split_ratio
        return pd.Series(train_mask, index=df.index), pd.Series(~train_mask, index=df.index)

def clean_text(text, lemmatize=False):
    """Продвинутая очистка текста"""
    if not isinstance(text, str) or pd.isna(text) or len(text.strip()) == 0:
        return ""

    # Базовая очистка
    text = text.lower()
    text = re.sub(r'<[^>]+>', '', text)  # Удаляем HTML
    text = re.sub(r'http\S+', '', text)  # Удаляем ссылки
    text = re.sub(r'[^\w\s]', ' ', text)  # Удаляем пунктуацию
    text = re.sub(r'\d+', ' ', text)  # Удаляем цифры
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы

    return text

# === ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ===
def load_and_merge_data():
    """Загрузка и объединение всех данных с продвинутой обработкой"""
    print("=" * 80)
    print("🚀 ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ")
    print("=" * 80)

    start_time = time.time()

    # Распаковка архивов
    zip_files = glob.glob('*.zip')
    if zip_files:
        print(f"📦 Найдено zip архивов: {len(zip_files)}")
        for zip_file in zip_files:
            print(f"   Распаковываем: {zip_file}")
            try:
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall('.')
                print(f"   ✅ Успешно распакован: {zip_file}")
            except Exception as e:
                print(f"   ❌ Ошибка при распаковке {zip_file}: {str(e)}")

    # Поиск файлов
    required_files = {
        'train': ['train.csv', 'train_data.csv'],
        'test': ['test.csv', 'test_data.csv'],
        'books': ['books.csv', 'books_data.csv'],
        'users': ['users.csv', 'users_data.csv'],
        'descriptions': ['book_descriptions.csv', 'descriptions.csv'],
        'genres': ['book_genres.csv', 'genres.csv']
    }

    file_paths = {}
    optional_files = ['genres']

    for key, filenames in required_files.items():
        found = False
        for filename in filenames:
            try:
                file_paths[key] = find_file(filename)
                print(f"✅ Найден файл для {key}: {filename}")
                found = True
                break
            except FileNotFoundError:
                continue

        if not found:
            if key in optional_files:
                print(f"⚠️  Не найден опциональный файл для {key}. Продолжаем без него.")
                file_paths[key] = None
            else:
                raise FileNotFoundError(f"Критический файл не найден для {key}. Проверьте данные.")

    # Автоопределение разделителя
    def detect_delimiter(file_path):
        """Автоопределение разделителя CSV"""
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                first_line = f.readline()

            if ';' in first_line and ',' in first_line:
                if first_line.count(';') > first_line.count(','):
                    return ';'
            elif ';' in first_line:
                return ';'
            return ','
        except Exception as e:
            print(f"⚠️  Ошибка при определении разделителя: {str(e)}. Используем ','")
            return ','

    # Загрузка данных
    def load_csv_safely(file_path, desc=""):
        """Безопасная загрузка CSV с обработкой ошибок"""
        if not file_path:
            return pd.DataFrame()

        sep = detect_delimiter(file_path)
        print(f"   Загружаем {Path(file_path).name} с разделителем '{sep}'")

        try:
            # Сначала пробуем стандартную загрузку
            df = pd.read_csv(file_path, sep=sep, low_memory=False)
            print(f"   ✅ {desc} shape: {df.shape}")
            return df
        except Exception as e:
            print(f"   ❌ Ошибка загрузки {Path(file_path).name}: {str(e)}")
            # Попытка загрузить с пропуском ошибок
            try:
                df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip', low_memory=False)
                print(f"   ⚠️  Загружено с пропуском ошибочных строк. Shape: {df.shape}")
                return df
            except Exception as e2:
                print(f"   ❌ Полный провал загрузки: {str(e2)}")
                return pd.DataFrame()

    # Загружаем основные данные
    train_df = load_csv_safely(file_paths['train'], "Train")
    test_df = load_csv_safely(file_paths['test'], "Test")

    # Фильтрация по has_read
    if 'has_read' in train_df.columns:
        original_len = len(train_df)
        train_df = train_df[train_df['has_read'] == 1].copy()
        print(f"   Отфильтровано по has_read=1: {original_len - len(train_df)} записей")

    # Загружаем дополнительные данные
    books_df = load_csv_safely(file_paths['books'], "Books")
    users_df = load_csv_safely(file_paths['users'], "Users")
    desc_df = load_csv_safely(file_paths['descriptions'], "Descriptions")
    genres_df = load_csv_safely(file_paths['genres'], "Genres") if file_paths['genres'] else None

    # Объединение данных
    print("\n🔗 Объединяем данные...")

    # Добавляем источник
    train_df['_source'] = 'train'
    test_df['_source'] = 'test'

    # Объединяем train и test
    combined = pd.concat([train_df, test_df], ignore_index=True)

    # Мерджим с пользователями
    print("   → Пользователи")
    if 'user_id' in users_df.columns and 'user_id' in combined.columns:
        combined = combined.merge(users_df, on='user_id', how='left')
        print(f"      После мержа: {combined.shape}")

    # Мерджим с книгами
    print("   → Книги")
    if 'book_id' in books_df.columns and 'book_id' in combined.columns:
        books_df = books_df.drop_duplicates(subset=['book_id'])
        combined = combined.merge(books_df, on='book_id', how='left')
        print(f"      После мержа: {combined.shape}")

    # Оптимизация памяти
    combined = reduce_mem_usage(combined, aggressive=True)

    print(f"✅ Общее время загрузки: {time.time() - start_time:.1f} сек")
    print(f"✅ Финальный размер объединенного датасета: {combined.shape}")

    return combined, desc_df, genres_df, train_df, test_df

# === ГЕНЕРАЦИЯ ПРОДВИНУТЫХ ФИЧЕЙ ===
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """Продвинутые временные признаки с обработкой ошибок"""
    if not config.ENABLE_TEMPORAL_FEATURES or constants.COL_TIMESTAMP not in df.columns:
        return df

    print("⏱️  Генерируем продвинутые временные признаки...")

    # Преобразуем в datetime с защитой
    if not pd.api.types.is_datetime64_any_dtype(df[constants.COL_TIMESTAMP]):
        try:
            df[constants.COL_TIMESTAMP] = pd.to_datetime(
                df[constants.COL_TIMESTAMP],
                errors='coerce',
                infer_datetime_format=True,
                dayfirst=True
            )
        except Exception as e:
            print(f"⚠️  Ошибка преобразования timestamp: {str(e)}")
            # Создаем искусственный timestamp
            df[constants.COL_TIMESTAMP] = pd.date_range(
                start='2020-01-01',
                periods=len(df),
                freq='D'
            )

    # Заполняем пропуски
    if df[constants.COL_TIMESTAMP].isnull().any():
        median_time = df[constants.COL_TIMESTAMP].median()
        null_count = df[constants.COL_TIMESTAMP].isnull().sum()
        print(f"⚠️  Заполняем {null_count} пропусков в timestamp: {median_time}")
        df[constants.COL_TIMESTAMP] = df[constants.COL_TIMESTAMP].fillna(median_time)

    # Извлекаем компоненты
    try:
        df['hour'] = df[constants.COL_TIMESTAMP].dt.hour
        df['day'] = df[constants.COL_TIMESTAMP].dt.day
        df['month'] = df[constants.COL_TIMESTAMP].dt.month
        df['year'] = df[constants.COL_TIMESTAMP].dt.year
        df['dayofweek'] = df[constants.COL_TIMESTAMP].dt.dayofweek
        df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(np.int8)
    except Exception as e:
        print(f"⚠️  Ошибка при извлечении временных компонентов: {str(e)}")
        # Добавляем базовые признаки
        df['hour'] = 12
        df['dayofweek'] = 3
        df['month'] = 6
        df['is_weekend'] = 0

    # Периоды дня с безопасной конвертацией
    try:
        time_period = pd.cut(
            df['hour'].fillna(12),
            bins=[-1, 6, 12, 18, 24],
            labels=[0, 1, 2, 3],
            include_lowest=True
        )
        df['time_period'] = safe_to_int(time_period, fill_value=1)
    except Exception as e:
        print(f"⚠️  Ошибка при создании time_period: {str(e)}")
        df['time_period'] = 1

    # Сезоны
    try:
        df['month_filled'] = df['month'].fillna(6)
        df['season'] = ((df['month_filled'] - 1) // 3 + 1).astype(np.int8)
        df.drop(columns=['month_filled'], inplace=True)
    except Exception as e:
        print(f"⚠️  Ошибка при создании seasons: {str(e)}")
        df['season'] = 2

    print(f"✅ Добавлено временных фич: {len([c for c in df.columns if 'time' in c or 'hour' in c or 'day' in c or 'month' in c or 'year' in c])}")
    return df

def add_target_encoding(df: pd.DataFrame, train_split: pd.DataFrame) -> pd.DataFrame:
    """Продвинутое target encoding с защитой от утечек и пропусков"""
    if not config.ENABLE_TARGET_ENCODING or train_split is None or constants.COL_TARGET not in train_split.columns:
        return df

    print("🎯 Генерируем target encoding...")

    # Глобальное среднее
    global_mean = train_split[constants.COL_TARGET].mean()

    # Категории для кодирования
    encode_cols = []
    for col in ['author_id', 'publisher', 'language']:
        if col in df.columns and col in train_split.columns:
            if train_split[col].nunique() > 1:  # Проверяем, что есть вариации
                encode_cols.append(col)

    for col in encode_cols:
        print(f"   → {col}")

        try:
            # Агрегация только на train
            agg = train_split.groupby(col, dropna=False)[constants.COL_TARGET].agg(
                ['mean', 'count']
            ).reset_index()

            # Bayesian smoothing
            agg['smoothed_mean'] = (
                agg['mean'] * agg['count'] + global_mean * 10
            ) / (agg['count'] + 10)

            # Мерджим
            rename_cols = {'smoothed_mean': f'{col}_target_mean'}
            df = df.merge(
                agg[[col] + list(rename_cols.keys())].rename(columns=rename_cols),
                on=col,
                how='left'
            )

            # Заполняем пропуски
            for new_col in rename_cols.values():
                if new_col in df.columns:
                    fill_value = global_mean
                    null_count_before = df[new_col].isnull().sum()
                    df[new_col] = df[new_col].fillna(fill_value)
                    null_count_after = df[new_col].isnull().sum()

                    if null_count_after > 0:
                        print(f"   ⚠️  Осталось {null_count_after} пропусков в {new_col} после заполнения")

        except Exception as e:
            print(f"   ❌ Ошибка при обработке {col}: {str(e)}")

    return df

def add_graph_features(df, train_df):
    """Графовые признаки через ALS с обработкой ошибок"""
    return df  # Отключено для стабильности

def add_bert_embeddings(df, desc_df, train_books):
    """Генерация эмбеддингов через Sentence-BERT с обработкой ошибок"""
    return add_text_features(df, desc_df, train_books)  # Используем TF-IDF вместо BERT

def add_text_features(df: pd.DataFrame, desc_df: pd.DataFrame, train_books: list):
    """Добавляем TF-IDF фичи с продвинутой обработкой и обработкой ошибок"""
    print("🔤 Добавляем TF-IDF фичи...")

    if desc_df.empty or len(train_books) == 0:
        print("   ⚠️  Нет данных для TF-IDF")
        return df

    try:
        # Предобработка описаний
        desc_df = desc_df.copy()
        if 'description' in desc_df.columns:
            desc_df['description_clean'] = desc_df['description'].apply(clean_text)
        else:
            print("   ⚠️  Нет колонки 'description' в данных описаний")
            desc_df['description_clean'] = ""

        # Создаем карту описаний
        desc_map = dict(zip(desc_df['book_id'], desc_df['description_clean'].fillna("no description")))

        # Учим TF-IDF только на train книгах
        train_desc_df = desc_df[desc_df['book_id'].isin(train_books)].copy()

        if len(train_desc_df) < 10:
            print("   ⚠️  Недостаточно описаний для обучения TF-IDF")
            return df

        # Обучаем векторизатор
        tfidf = TfidfVectorizer(
            max_features=config.TFIDF_MAX_FEATURES,
            min_df=config.TFIDF_MIN_DF,
            max_df=config.TFIDF_MAX_DF,
            ngram_range=config.TFIDF_NGRAM_RANGE,
            stop_words=None,
            token_pattern=r'(?u)\b\w+\b',
            lowercase=False
        )

        # Обрабатываем ошибки при обучении
        try:
            tfidf.fit(train_desc_df['description_clean'])
            print(f"   Обучен TF-IDF с {len(tfidf.get_feature_names_out())} фичами")
        except Exception as e:
            print(f"   ❌ Ошибка при обучении TF-IDF: {str(e)}")
            return df

        # Применяем к описаниям всех книг
        all_books = df['book_id'].unique()
        all_descriptions = [desc_map.get(book_id, "no description") for book_id in all_books]

        # Обрабатываем ошибки при трансформации
        try:
            tfidf_matrix = tfidf.transform(all_descriptions)
        except Exception as e:
            print(f"   ❌ Ошибка при трансформации TF-IDF: {str(e)}")
            return df

        # Создаем DataFrame
        tfidf_df = pd.DataFrame(
            tfidf_matrix.toarray(),
            index=all_books,
            columns=[f'tfidf_{i}' for i in range(min(tfidf_matrix.shape[1], config.TFIDF_MAX_FEATURES))]
        ).reset_index().rename(columns={'index': 'book_id'})

        # Мерджим с основным датафреймом
        df = df.merge(tfidf_df, on='book_id', how='left')

        # Статистики
        tfidf_cols = [col for col in df.columns if col.startswith('tfidf_')]
        if tfidf_cols:
            df['tfidf_sum'] = df[tfidf_cols].sum(axis=1)
            df['tfidf_mean'] = df[tfidf_cols].mean(axis=1)
            df['tfidf_max'] = df[tfidf_cols].max(axis=1)

        print(f"✅ Добавлено {len(tfidf_cols)} TF-IDF фич + 3 статистики")
        return df

    except Exception as e:
        print(f"❌ Ошибка при генерации TF-IDF: {str(e)}")
        return df

def add_cluster_features(df, n_clusters=10):
    """Кластеризация для обнаружения паттернов с обработкой ошибок"""
    return df  # Отключено для стабильности

def add_aggregate_features(df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    """Продвинутые агрегатные фичи с обработкой ошибок"""
    print("📊 Генерируем продвинутые агрегатные фичи...")

    if train_df is None or train_df.empty:
        print("   ⚠️  Нет данных для генерации агрегатных фичей")
        return df

    # Фильтруем только train данные для расчета статистик
    train_data = train_df[train_df['_source'] == 'train'].copy()

    if train_data.empty:
        print("   ⚠️  Нет train данных для агрегации")
        return df

    # Агрегаты по пользователям
    print("   → Пользовательские агрегаты")
    try:
        user_agg = train_data.groupby('user_id', dropna=False).agg(
            user_mean_rating=(constants.COL_TARGET, 'mean'),
            user_ratings_count=(constants.COL_TARGET, 'count'),
            user_rating_std=(constants.COL_TARGET, 'std'),
        ).reset_index()

        # Заполняем NaN в std
        user_agg['user_rating_std'] = user_agg['user_rating_std'].fillna(0)
        df = df.merge(user_agg, on='user_id', how='left')
    except Exception as e:
        print(f"   ❌ Ошибка при генерации пользовательских агрегатов: {str(e)}")

    # Агрегаты по книгам
    print("   → Книжные агрегаты")
    try:
        book_agg = train_data.groupby('book_id', dropna=False).agg(
            book_mean_rating=(constants.COL_TARGET, 'mean'),
            book_ratings_count=(constants.COL_TARGET, 'count'),
            book_rating_std=(constants.COL_TARGET, 'std'),
        ).reset_index()

        # Заполняем NaN в std
        book_agg['book_rating_std'] = book_agg['book_rating_std'].fillna(0)
        df = df.merge(book_agg, on='book_id', how='left')
    except Exception as e:
        print(f"   ❌ Ошибка при генерации книжных агрегатов: {str(e)}")

    # Агрегаты по авторам
    if 'author_id' in train_data.columns and train_data['author_id'].notnull().any():
        print("   → Авторские агрегаты")
        try:
            author_agg = train_data.groupby('author_id', dropna=False).agg(
                author_mean_rating=(constants.COL_TARGET, 'mean'),
                author_ratings_count=(constants.COL_TARGET, 'count'),
                author_rating_std=(constants.COL_TARGET, 'std')
            ).reset_index()

            author_agg['author_rating_std'] = author_agg['author_rating_std'].fillna(0)
            df = df.merge(author_agg, on='author_id', how='left')
        except Exception as e:
            print(f"   ❌ Ошибка при генерации авторских агрегатов: {str(e)}")

    # Продвинутые взаимодействия
    if config.ENABLE_ADVANCED_INTERACTIONS:
        print("   → Продвинутые взаимодействия")

        try:
            # Разница между средней книгой и пользователем
            if 'book_mean_rating' in df.columns and 'user_mean_rating' in df.columns:
                df['rating_diff_user_book'] = df['book_mean_rating'] - df['user_mean_rating']

            # Нормализованная разница
            if 'rating_diff_user_book' in df.columns and 'user_rating_std' in df.columns and 'book_rating_std' in df.columns:
                df['rating_diff_normalized'] = (
                    df['rating_diff_user_book'] /
                    (df['user_rating_std'] + df['book_rating_std'] + 1e-6)
                ).astype(np.float32)

            # Отношение количества оценок
            if 'user_ratings_count' in df.columns and 'book_ratings_count' in df.columns:
                df['rating_count_ratio'] = (
                    np.log1p(df['user_ratings_count']) /
                    (np.log1p(df['book_ratings_count']) + 1)
                ).astype(np.float32)
        except Exception as e:
            print(f"   ❌ Ошибка при генерации взаимодействий: {str(e)}")

    # Логарифмические преобразования
    for col in ['user_ratings_count', 'book_ratings_count']:
        if col in df.columns:
            try:
                df[f'log_{col}'] = np.log1p(df[col]).astype(np.float32)
            except Exception as e:
                print(f"   ❌ Ошибка при логарифмическом преобразовании {col}: {str(e)}")

    print("✅ Генерация агрегатных фичей завершена")
    return df

def prepare_data_for_lgbm(df):
    """Подготовка данных для LightGBM - удаляем все нечисловые фичи"""
    print("🔧 Подготовка данных для LightGBM...")

    # Удаляем все строковые колонки
    str_cols = df.select_dtypes(include=['object']).columns.tolist()
    if str_cols:
        print(f"   Удаляем строковые колонки: {str_cols}")
        df = df.drop(columns=str_cols)

    # Удаляем временные колонки
    datetime_cols = df.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()
    if datetime_cols:
        print(f"   Удаляем временные колонки: {datetime_cols}")
        df = df.drop(columns=datetime_cols)

    # Удаляем категориальные колонки с большим количеством уникальных значений
    cat_cols_to_drop = []
    for col in df.select_dtypes(include=['category']).columns:
        if df[col].nunique() > 1000:  # Слишком много категорий
            cat_cols_to_drop.append(col)

    if cat_cols_to_drop:
        print(f"   Удаляем категориальные колонки с большим количеством уникальных значений: {cat_cols_to_drop}")
        df = df.drop(columns=cat_cols_to_drop)

    # Преобразуем оставшиеся категориальные колонки в числовой формат
    for col in df.select_dtypes(include=['category']).columns:
        df[col] = df[col].cat.codes

    # Заполняем оставшиеся пропуски
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'float32']:
                fill_value = df[col].median()
            else:
                fill_value = -1
            df[col] = df[col].fillna(fill_value)

    # Проверяем, что остались только числовые типы
    non_numeric = df.select_dtypes(exclude=np.number).columns.tolist()
    if non_numeric:
        print(f"   ⚠️  Все еще есть нечисловые колонки: {non_numeric}")
        df = df.drop(columns=non_numeric)

    print(f"✅ Подготовлено {len(df.columns)} числовых фич для LightGBM")
    return df

def handle_missing_values(df: pd.DataFrame, train_stats: dict = None):
    """Продвинутое заполнение пропусков с многоуровневой стратегией"""
    print("🧩 Продвинутое заполнение пропусков...")

    # 1. Заполняем пропуски в числовых колонках
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    for col in num_cols:
        if col in df.columns and df[col].isnull().any():
            null_count = df[col].isnull().sum()

            # Стратегия заполнения зависит от колонки
            fill_value = None

            if train_stats and col in train_stats:
                fill_value = train_stats[col]
            elif col in ['user_mean_rating', 'book_mean_rating', 'author_mean_rating']:
                fill_value = df[constants.COL_TARGET].mean() if constants.COL_TARGET in df.columns else 5.0
            elif 'count' in col.lower() or 'rating_count' in col.lower():
                fill_value = 0
            elif 'std' in col.lower():
                fill_value = 0
            elif 'diff' in col.lower() or 'ratio' in col.lower():
                fill_value = 0
            elif col in ['age', 'publication_year']:
                fill_value = df[col].median() if df[col].notnull().any() else 35
            else:
                fill_value = df[col].median() if df[col].notnull().any() else 0

            df[col] = df[col].fillna(fill_value)
            new_null_count = df[col].isnull().sum()

            if new_null_count > 0:
                print(f"   ⚠️  Осталось {new_null_count} пропусков в {col} после заполнения")
                # Принудительное заполнение остатков
                df[col] = df[col].fillna(fill_value)

    # 2. Заполняем пропуски в категориальных колонках
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    for col in cat_cols:
        if col in df.columns and df[col].isnull().any():
            fill_value = constants.MISSING_CAT_VALUE
            null_count = df[col].isnull().sum()
            df[col] = df[col].fillna(fill_value)
            print(f"   → {col}: заполнено {null_count} пропусков значением '{fill_value}'")

    # 3. Финальная проверка и принудительное заполнение
    remaining_nulls = df.isnull().sum()
    remaining_nulls = remaining_nulls[remaining_nulls > 0]

    if len(remaining_nulls) > 0:
        print(f"   ⚠️  Осталось пропусков после обработки: {remaining_nulls.to_dict()}")
        # Принудительно заполняем все оставшиеся пропуски
        for col in remaining_nulls.index:
            if df[col].dtype in ['float64', 'float32', 'int64', 'int32', 'int16', 'int8']:
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(constants.MISSING_CAT_VALUE)
        print("   ✅ Все оставшиеся пропуски принудительно заполнены")

    return df

# === МОДЕЛИРОВАНИЕ ===
def train_model(X_train, y_train, X_val, y_val, cat_features):
    """Обучение LightGBM модели с предварительной подготовкой данных"""
    print("🤖 Обучение модели LightGBM...")

    try:
        # Подготавливаем данные для LightGBM
        X_train_prepared = prepare_data_for_lgbm(X_train.copy())
        X_val_prepared = prepare_data_for_lgbm(X_val.copy())

        # Оставляем только общие фичи
        common_features = list(set(X_train_prepared.columns) & set(X_val_prepared.columns))
        X_train_prepared = X_train_prepared[common_features]
        X_val_prepared = X_val_prepared[common_features]

        print(f"✅ Используем {len(common_features)} фич для обучения")

        # LightGBM
        train_data = lgb.Dataset(
            X_train_prepared,
            label=y_train,
            free_raw_data=False
        )

        val_data = lgb.Dataset(
            X_val_prepared,
            label=y_val,
            reference=train_data,
            free_raw_data=False
        )

        # Обучение
        model = lgb.train(
            config.LGB_PARAMS,
            train_data,
            valid_sets=[train_data, val_data],
            valid_names=['train', 'val'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=config.EARLY_STOPPING_ROUNDS),
                lgb.log_evaluation(period=50),
            ]
        )

        return model, common_features

    except Exception as e:
        print(f"❌ Ошибка при обучении LightGBM: {str(e)}")
        raise

def adaptive_calibration(preds, y_true=None, strategy='median'):
    """Адаптивная калибровка предсказаний"""
    print("🎯 Применяем адаптивную калибровку...")

    preds = np.array(preds).copy()
    original_median = np.median(preds)

    if strategy == 'median' and y_true is not None:
        target_median = np.median(y_true)
        shift = target_median - original_median
        preds += shift * 0.8  # Частичное смещение
        print(f"   Сдвиг медианы: {shift:.4f} (цель: {target_median:.2f})")

    # Ограничение диапазона
    preds = np.clip(preds, constants.PREDICTION_MIN_VALUE + 0.5, constants.PREDICTION_MAX_VALUE - 0.5)

    new_median = np.median(preds)
    print(f"   Медиана до калибровки: {original_median:.4f} → после: {new_median:.4f}")

    return preds

# === ОСНОВНОЙ ПАЙПЛАЙН ===
def main_pipeline():
    """Основной пайплайн с продвинутыми фичами и обработкой ошибок"""
    print("\n" + "=" * 80)
    print("🚀 ЗАПУСК СУПЕР-ПАЙПЛАЙНА")
    print(f"   Время запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    start_time = time.time()

    try:
        # Загрузка данных
        combined, desc_df, genres_df, original_train_df, test_df = load_and_merge_data()

        # Добавление базовых фичей
        combined = add_temporal_features(combined)

        # Получаем список train книг
        train_books = original_train_df['book_id'].unique() if not original_train_df.empty else []

        # Текстовые фичи
        combined = add_text_features(combined, desc_df, train_books)

        # Разделение на train и test
        train_processed = combined[combined['_source'] == 'train'].copy()
        test_processed = combined[combined['_source'] == 'test'].copy()

        # Временной сплит
        print("\n⏱️  Выполняем временной сплит...")
        train_mask, val_mask = temporal_split_by_date(train_processed, config.TEMPORAL_SPLIT_RATIO)

        train_split = train_processed.loc[train_mask].copy()
        val_split = train_processed.loc[val_mask].copy()

        print(f"   Train split: {len(train_split):,} записей")
        print(f"   Val split: {len(val_split):,} записей")

        # Генерация продвинутых фичей
        print("\n✨ Генерация продвинутых фичей для train...")
        train_split = add_aggregate_features(train_split, train_split)
        train_split = add_target_encoding(train_split, train_split)

        # Статистики для заполнения пропусков
        train_stats = {}
        for col in train_split.select_dtypes(include=np.number).columns:
            if col != constants.COL_TARGET:
                train_stats[col] = train_split[col].median()

        # Заполнение пропусков
        train_split = handle_missing_values(train_split, train_stats)
        val_split = add_aggregate_features(val_split, train_split)
        val_split = add_target_encoding(val_split, train_split)
        val_split = handle_missing_values(val_split, train_stats)

        # Оптимизация памяти
        print("\n💾 Оптимизация памяти...")
        train_split = reduce_mem_usage(train_split, aggressive=True)
        val_split = reduce_mem_usage(val_split, aggressive=True)
        train_processed = reduce_mem_usage(train_processed, aggressive=True)

        # Проверка наличия target колонки
        if constants.COL_TARGET not in train_split.columns:
            if 'rating' in train_split.columns:
                train_split[constants.COL_TARGET] = train_split['rating']
                val_split[constants.COL_TARGET] = val_split['rating']
                print("⚠️  Используем колонку 'rating' как target")
            else:
                raise ValueError(f"Не найдена колонка target: {constants.COL_TARGET}")

        # Подготовка данных
        exclude_cols = [
            '_source', 'timestamp', 'has_read', 'title', 'author_name', 'rating',
            'description', 'description_clean', 'book_description'
        ]

        features = [col for col in train_split.columns if col not in exclude_cols]

        # Подготовка данных
        X_train = train_split[features]
        y_train = train_split[constants.COL_TARGET]
        X_val = val_split[features]
        y_val = val_split[constants.COL_TARGET]

        # Обучение модели
        print("\n" + "=" * 80)
        print("🤖 ОБУЧЕНИЕ МОДЕЛИ")
        print("=" * 80)

        model, used_features = train_model(X_train, y_train, X_val, y_val, [])

        # Оценка качества
        X_val_prepared = prepare_data_for_lgbm(X_val.copy())
        X_val_prepared = X_val_prepared[used_features]
        val_preds = model.predict(X_val_prepared)

        # Адаптивная калибровка
        val_preds = adaptive_calibration(val_preds, y_val, strategy='median')

        # Метрики
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        mae = mean_absolute_error(y_val, val_preds)
        score = 1 - ((rmse/10) + (mae/10)) / 2

        print(f"\n📊 Результаты на валидации:")
        print(f"   RMSE: {rmse:.5f}")
        print(f"   MAE: {mae:.5f}")
        print(f"   SCORE: {score:.5f}")
        print(f"   Медиана предсказаний: {np.median(val_preds):.2f}")
        print(f"   Среднее предсказаний: {val_preds.mean():.2f}")

        # Важность фичей
        try:
            feature_importance = pd.DataFrame({
                'feature': used_features,
                'importance': model.feature_importance('gain')
            }).sort_values('importance', ascending=False)

            feature_importance.to_csv(config.FEATURE_IMPORTANCE_PATH, index=False)
            print(f"✅ Важность фичей сохранена: {config.FEATURE_IMPORTANCE_PATH}")
            print("\n🔑 Топ-10 важных фичей:")
            for _, row in feature_importance.head(10).iterrows():
                print(f"   {row['feature']}: {row['importance']:.0f}")
        except Exception as e:
            print(f"⚠️  Ошибка при расчете важности фичей: {str(e)}")

        # Предсказание на тесте
        print("\n" + "=" * 80)
        print("🔮 ПРЕДСКАЗАНИЕ НА ТЕСТОВЫХ ДАННЫХ")
        print("=" * 80)

        # Подготовка тестовых данных
        test_processed = add_aggregate_features(test_processed, train_processed[train_processed['_source'] == 'train'])
        test_processed = add_target_encoding(test_processed, train_split)
        test_processed = handle_missing_values(test_processed, train_stats)

        # Фильтрация фичей
        X_test = test_processed[features]

        # Предсказание
        print("   Генерируем предсказания...")
        X_test_prepared = prepare_data_for_lgbm(X_test.copy())
        X_test_prepared = X_test_prepared[used_features]
        test_preds = model.predict(X_test_prepared)

        # Продвинутая калибровка
        test_preds = adaptive_calibration(test_preds, y_train, strategy='median')

        # Статистика
        print(f"\n📈 Статистика финальных предсказаний:")
        print(f"   Медиана: {np.median(test_preds):.4f}")
        print(f"   Среднее: {test_preds.mean():.4f}")
        print(f"   Min: {test_preds.min():.4f}")
        print(f"   Max: {test_preds.max():.4f}")
        print(f"   Std: {test_preds.std():.4f}")

        # Сохранение результатов
        submission = test_df[['user_id', 'book_id']].copy()
        submission['rating_predict'] = test_preds

        # Округление
        submission['rating_predict'] = submission['rating_predict'].round(2)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"submission_{timestamp}_{score:.5f}.csv"
        submission.to_csv(filename, index=False)
        print(f"\n✅ Сабмит сохранен: {filename}")
        print(f"   Размер: {submission.shape}")

        # Сохранение модели
        try:
            model.save_model(f"{config.MODEL_FILENAME}_{timestamp}.txt")
            print(f"✅ Модель сохранена: {config.MODEL_FILENAME}_{timestamp}.txt")
        except Exception as e:
            print(f"⚠️  Ошибка при сохранении модели: {str(e)}")

        # Общее время
        total_time = time.time() - start_time
        print(f"\n{'=' * 80}")
        print(f"🏁 ПАЙПЛАЙН ЗАВЕРШЕН ЗА {total_time:.1f} СЕКУНД")
        print(f"🎯 Финальный скор: {score:.5f}")
        print("=" * 80)

        return score, submission, model

    except Exception as e:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: {str(e)}")
        import traceback
        traceback.print_exc()

        # Создаем резервный сабмит
        print("\n💾 Создаем резервный сабмит...")
        try:
            if 'test_df' in locals() and test_df is not None:
                submission = test_df[['user_id', 'book_id']].copy()
                submission['rating_predict'] = 5.5  # Нейтральное значение

                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                backup_filename = f"submission_backup_{timestamp}.csv"
                submission.to_csv(backup_filename, index=False)
                print(f"✅ Резервный сабмит сохранен: {backup_filename}")
                return 0.5, submission, None
            else:
                print("⚠️  Не удалось создать резервный сабмит - нет test_df")
                return 0.5, None, None
        except Exception as e2:
            print(f"❌ Ошибка при создании резервного сабмита: {str(e2)}")
            return 0.5, None, None

# === ЗАПУСК ===
if __name__ == "__main__":
    main_pipeline()

Training until validation scores don't improve for 100 rounds
[50]	train's rmse: 2.42504	val's rmse: 2.888
[100]	train's rmse: 2.19031	val's rmse: 2.87137
[150]	train's rmse: 2.0861	val's rmse: 2.89194
[200]	train's rmse: 2.03675	val's rmse: 2.91703
Early stopping, best iteration is:
[104]	train's rmse: 2.17934	val's rmse: 2.86975
